In [45]:
#r "nuget:Microsoft.ML.AutoML, 0.17.0-preview"

In [81]:
using Microsoft.DotNet.Interactive;

using System.IO;
using System.Collections.Generic;
using System.IO.Compression;

#r "nuget:FSharp.Core"
//#r "nuget:FSharp.Compiler.Private.Scripting"

static class PackagesHelper
{
    static string GetPathRelativeToPackages(string file, DirectoryInfo directory)
    {
        var fileName = file.Replace("/", @"\");
        var strippedRoot = fileName.Replace(directory.ToString().Replace("/", @"\"), "");
        var relativePath = Path.Combine(Trim(directory.Parent.Name), Trim(directory.Name), Trim(strippedRoot));
        return relativePath;

        string Trim(string path)
        {
            return path.Trim(Path.DirectorySeparatorChar, Path.AltDirectorySeparatorChar);
        }
    }

    static void ZipInsertFileFrom(ZipArchiveEntry entry, FileInfo file)
    {
        using (StreamWriter writer = new StreamWriter(entry.Open()))
        {
            using (StreamReader reader = new StreamReader(file.Open(FileMode.Open)))
            {
                while(!reader.EndOfStream)
                {
                    var b = reader.Read();
                    writer.Write(b);
                }
            }
        }
    }

    public static FileInfo MakePackageZipFile(FileInfo zipPath, IEnumerable<DirectoryInfo> directories)
    {
        using (FileStream zipToOpen = new FileStream(zipPath.ToString(), FileMode.Create))
        {
            using (ZipArchive archive = new ZipArchive(zipToOpen, ZipArchiveMode.Create))
            {
                foreach (var directory in directories)
                {
                    foreach (var file in directory.EnumerateFiles("*", SearchOption.AllDirectories))
                    {
                        // Strip off and make path relative
                        var nugetPath = GetPathRelativeToPackages(file.FullName, directory);
                        ZipArchiveEntry entry = archive.CreateEntry(nugetPath);
                        ZipInsertFileFrom(entry, file);
                    }
                }
            }
        }
        return zipPath;
    }
    
    public static FileInfo ZipPackages(DirectoryInfo workDir)
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var directories = restoreContext.ResolvedPackageReferences.Select(r => r.PackageRoot);
        return MakePackageZipFile(new FileInfo(Path.Combine(workDir.ToString(), "packages.zip")), directories);
    }

    public static void UnzipPackages(FileInfo zipFile, DirectoryInfo extractPath)
    {
         ZipFile.ExtractToDirectory(zipFile.ToString(), extractPath.ToString());
    }
    
    // This returns the full paths to the packagemanager library to be deployed with compiler scripts
    // HardCoded to:
    //     FSharp.Core.dll
    //     Interactive.DependencyManager.dll   ---> this name will definitely change. to Microsoft.DotNet.DependencyManager.dll
    public static IEnumerable<FileInfo> GetPathsToPackageManagerLibraries()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.AssemblyPaths)
            {
                var p = path.ToString();
                if (p.EndsWith("FSharp.Core.dll")) yield return path;
                else if (p.EndsWith("Microsoft.DotNet.DependencyManager.dll")) yield return path;
                else if (p.EndsWith("Interactive.DependencyManager.dll")) yield return path;
            }
        }
    }

    // This returns the relative paths to the referenced dll's
    public static IEnumerable<string> GetRelativeAssemblyProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.AssemblyPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }

    // This returns the full paths to the referenced dll's
    public static IEnumerable<string> GetRelativeNativeProbingPaths()
    {
        var restoreContext = KernelInvocationContext.Current.HandlingKernel as IPackageRestoreContext;
        var packages = restoreContext.ResolvedPackageReferences;
        foreach (var package in packages)
        {
            foreach (var path in package.ProbingPaths)
            {
                yield return GetPathRelativeToPackages(path.FullName, package.PackageRoot);
            }
        }
    }
}

In [82]:
var workDir = new DirectoryInfo(Path.Combine(Path.GetTempPath(), Path.GetRandomFileName()));
if (!workDir.Exists) workDir.Create();

var zipFile = PackagesHelper.ZipPackages(workDir);

//var dllPaths = PackagesHelper.GetPathsToPackageManagerLibraries();

// Add these to deployment

// Assembly probing paths
var assemblyProbingPathsFile = new FileInfo(Path.Combine(workDir.FullName, "assemblyProbes.probe"));
var assemblyProbingPaths = PackagesHelper.GetRelativeAssemblyProbingPaths();
File.WriteAllLines(assemblyProbingPathsFile.FullName, assemblyProbingPaths);

// Native probing paths
var nativeProbingPathsFile = new FileInfo(Path.Combine(workDir.FullName, "nativeProbes.probe"));
var nativeProbingPaths = PackagesHelper.GetRelativeNativeProbingPaths();
File.WriteAllLines(nativeProbingPathsFile.FullName, nativeProbingPaths);

Console.WriteLine($"{workDir}");

C:\Users\kevinr\AppData\Local\Temp\eyimfeyg.tvs


In [ ]:
using Microsoft.DotNet.Interactive;

using System.IO;
using System.Collections.Generic;
using System.IO.Compression;

#r "nuget:FSharp.Core"
//#r "nuget:FSharp.Compiler.Private.Scripting"

static class DeployedPackageHelper
{
    public static void UnzipPackages(FileInfo zipFile, DirectoryInfo extractPath)
    {
         ZipFile.ExtractToDirectory(zipFile.ToString(), extractPath.ToString());
    }

    public static Deploy

}